# COVID Government Response Paper Data Analysis

# TO DO:



- [ ] Continue to update this **[file](https://1drv.ms/x/s!AjWX5HOdYY23kf9x5S7g8LKLGlseVg?e=992nsi)** of data source locations 
- [ ] Figure out what to do about the negative values for case counts and death counts
- [ ] Continue to review the tutorial here [How to run a Zero-inflated Model with Random Effects](https://stats.idre.ucla.edu/sas/faq/how-do-i-run-a-random-effect-zero-inflated-poisson-model-using-nlmixed/)
- [ ] Figure out how to lag the case and death vars
- [ ] fill forward all of the control measures vars for all of the countries with data on these vars
- [ ] formalize the sensitivity analysis for the different thresholds for classifying SARS exp

### Completed:
- [X] Need to explore the missingness of the Oxford data. Sort the countries by GDP and examine what the missingness matrix looks like. **If you could run imputation on this data then you would have a major leg up on the other paper working on the similar topic. (on to of the other benefits to your paper)**

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] =15 ,9
import seaborn as sns
from matplotlib.pyplot import figure

import numpy as np

import datetime

import os
from pathlib import Path
import shutil
import missingno as msno
import re
from glob import glob

In [ ]:
data_path = '..\Modified Data Sets'
control_data_path = '..\Control Data'
graphics_path = '..\graphics'

In [ ]:
#importing main data set
df_path = glob(f'{data_path}\Final COVID Data Set (Through*.xlsx')[0]
df = pd.read_excel(df_path)
df = df.loc[(~df.date.isnull())]

#Creating data set to analyze missingness
no_dup = df.copy()
no_dup = no_dup.loc[~no_dup.case_count.isnull()]
no_dup['dup'] = no_dup.duplicated(['country'],keep='last')
miss_anal = no_dup.loc[~no_dup.dup]
miss_anal = miss_anal.loc[~miss_anal.quarantine_efficiency.isnull()]
#Creating dataset to analyze the overall trend
df_indexed = (df.groupby(by='date').sum()).filter(items=['case_count','death_count'])
df_indexed = df.set_index('date')

In [ ]:
df.columns

In [ ]:
df.info()

## Looking at Negative Values

In [ ]:
plt.axhline()

In [ ]:
df.loc[df.country == 'USA']

In [ ]:
df.country.unique()

In [ ]:
country = 'france'
var = 'Case Count'
y_var1 = 'case_rol_mean7'
y_var2 = 'death_rol_mean7'
c = df.loc[df.country.str.contains(country,re.IGNORECASE)]
ax = sns.lineplot( x='date',y=y_var1,data=c,label=y_var1)
ax2 = ax.twinx()
sns.lineplot( x='date',y=y_var2,data=c, ax=ax2, color='black', label=y_var2)
plt.xlabel('Date')
plt.ylabel(var)
plt.axhline(0, c='black')
plt.title(f'County-level Daily COVID-19 {var} for {c.country.unique()[0][0].upper()}{c.country.unique()[0][1:].lower()}')

In [ ]:

var = '7-Day Rolling Mean'
c = df.loc[df.country.str.contains(country,re.IGNORECASE)]
sns.lineplot( x='date',y=y_var,data=c,label=var)
plt.xlabel('Date')
plt.ylabel(var)
plt.axhline(0, c='black')
plt.title(f'County-level Daily COVID-19 {var} for {c.country.unique()[0][0].upper()}{c.country.unique()[0][1:].lower()}')

> ### Evaluating Stationality

In [ ]:
# from statsmodels.tsa.stattools import adfuller
# dftest = adfuller(df_indexed.case_count,autolag='AIC')
# dfoutput = pd.Series(dftest[0:4], index = ['Test Statistic', 'p-value', '#Lags Used', 'n'])
# for key,value in dftest[4].items():
#     dfoutput[f'Critical Value ({key})']= value
# print(dfoutput)

In [ ]:
sns.lineplot(x = 'date', y ='case_count', hue = 'country', data = df,legend=False)

In [ ]:
sns.lineplot(x = 'date', y ='death_count', hue = 'country', data = df,legend=False)

In [ ]:
rolmean = df_indexed.case_count.rolling(window=7).mean()
rolstd = df_indexed.case_count.rolling(window=7).std()
rolmean.plot()
rolstd.plot()
plt.title('Global Case Count Rolling Mean & STD')

In [ ]:
rolmean = df_indexed.death_count.rolling(window=7).mean()
rolstd = df_indexed.death_count.rolling(window=7).std()
mean = plt.plot(rolmean, label='Rolling Mean')
std = plt.plot(rolstd, label='Rolling STD')
plt.legend(loc='best')
plt.title('Global Death Count Rolling Mean & STD')
plt.show()

In [ ]:
# rolmean = df_indexed.death_count.rolling(window=7).mean()

# rolstd = df_indexed.death_count.rolling(window=7).std()
# rolmean.plot( label='Rolling Mean')
# rolstd.plot( label='Rolling STD')

# # death = plt.plot(df_indexed.death_count, label = 'Death Counts')
# plt.legend(loc='best')
# plt.title('Rolling Mean & STD')
# # rolstd.plot()
# plt.show()

### Missingness Analysis

>> ##### Missingness Sorted by GDP Rank

In [ ]:
miss_anal.sort_values(by='gdp_rank',inplace=True)
df_nomiss = miss_anal.loc[~miss_anal.case_count.isnull()].copy()
msno.matrix(df_nomiss, color=(0.0, 0.10, 0.00))

In [ ]:
df_nomiss_cc = df_nomiss.loc[~df_nomiss.school_close.isnull()].copy()
msno.matrix(df_nomiss_cc, color=(0.0, 0.30, 0.00))


>> ##### Missingness Sorted by Population

In [ ]:
msno.bar(df_nomiss, color=(0.0, 0.00, 0.90))

In [ ]:
msno.heatmap(df_nomiss)

In [ ]:
msno.dendrogram(df_nomiss)

## Descriptive Analysis

In [ ]:
df.case_count.describe()

In [ ]:
df.death_count.describe()

In [ ]:
df.info()

In [ ]:
sns.set_style('ticks')
sns.pairplot(df, vars= ['case_count','pop_2020', 'mers_sars_sum','school_close','large_gather','internat_travel','public_events'])

In [ ]:
sns.pairplot(df, vars= ['death_count','pop_2020', 'gdp_rank','ages_65_and_above_of_total_population','public_events'])

>## Distribution Analysis

In [ ]:
sns.set_style('whitegrid')
sns.distplot(df.case_count)
plt.title('Distribution of Global Daily COVID-19 Case Counts')
plt.rcParams['figure.figsize'] = (25,10) # changes plot size
plt.xlabel('Case Counts')

In [ ]:
sns.distplot(df.death_count)
plt.title('Distribution of Global Daily COVID-19 Case Counts')
plt.rcParams['figure.figsize'] = (25,10) # changes plot size
plt.xlabel('Case Counts')

# Working on the Fitted values for the Irma Arbo Project

In [ ]:
irma_dist_path = r"C:\Users\chacr\OneDrive\USF\Misc Research\GA Work\Data Analysis\Final Analysis SAS Data sets\COUNT DATASET\Data sets\Irma Eye Distance 2-8-20.xlsx"
fitted_path = r"C:\Users\chacr\OneDrive\USF\Misc Research\GA Work\Data Analysis\Final Analysis SAS Data sets\COUNT DATASET\Exported Datasets\Fitted Values For Model(Var Storm Dist)11-1-20.xlsx"
irma= pd.read_excel(irma_dist_path)
irma.columns = irma.columns.str.lower()
fitted = pd.read_excel(fitted_path)
fitted.columns = fitted.columns.str.lower()
irma['sed_km'] = round(irma.storm_eye_dist/1000,3)
irma.sort_values(by='sed_km')
irma_fitted = irma.merge(fitted,left_on='sed_km',right_on='storm_eye_dist_km')
irma_fitted.drop(columns=['storm_eye_dist_km','stmtno'],inplace=True)
for dists in distlist:
    print(f'lsmeans post_h /at STORM_EYE_DIST_KM =({dists}) ilink diff;')
distlist = list(irma.sed_km)

In [ ]:
fitted

In [ ]:
irma_fitted.columns

In [ ]:
irma_fitted.to_csv(r"C:\Users\chacr\OneDrive\USF\Misc Research\GA Work\Data Analysis\Final Analysis SAS Data sets\COUNT DATASET\Exported Datasets\Fitted Values & Storm Distance.csv", index=False)

In [ ]:
sns.lineplot(x='sed_km', y='mu_pre', data =irma_fitted)
sns.lineplot(x='sed_km', y='mu_post', data =irma_fitted)

# Modeling Analysis

>> Model specification

\begin{equation}
\tag{2.13}
stack.loss_i = \alpha_n + \beta air_i + e_i, \text{ where } e_i \sim \text{N}(0,\sigma^2) 
\end{equation}

\begin{equation}
\tag{2.7}
\begin{bmatrix}stack.loss_1\\stack.loss_2\\stack.loss_3\\stack.loss_4\end{bmatrix}
= 
\begin{bmatrix}
\alpha&\beta&0&0&0\\
\alpha&0&\beta&0&0\\
\alpha&0&0&\beta&0\\
\alpha&0&0&0&\beta
\end{bmatrix}
\begin{bmatrix}1\\air_1\\air_2\\air_3\\air_4\end{bmatrix}
+
\begin{bmatrix}e_1\\e_2\\e_3\\e_4\end{bmatrix}
\end{equation}

\[\begin{equation}\tag{2.7}\begin{bmatrix}stack.loss_1\\stack.loss_2\\stack.loss_3\\stack.loss_4\end{bmatrix}=\begin{bmatrix}\alpha&\beta&0&0&0\\ \alpha&0&\beta&0&0\\ \alpha&0&0&\beta&0\\ \alpha&0&0&0&\beta\end{bmatrix}
\end{equation}]